<img style="float: left;padding-right: 10px" width ="40px" src="https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/main/Images/SemanticKernelLogo.png">

## Semantic Kernel - Optimizing with Decision Frames  

Work in Progress 

### Step 1 - Initialize Configuration Builder & Build the Semantic Kernel Orchestration 

In [1]:
// Import the required NuGet configuration packages
#r "nuget: Microsoft.Extensions.Configuration, 9.0.0"
#r "nuget: Microsoft.Extensions.Configuration.Json, 9.0.0"

using Microsoft.Extensions.Configuration.Json;
using Microsoft.Extensions.Configuration;
using System.IO;

// Load the configuration settings from the local.settings.json and secrets.settings.json files
// The secrets.settings.json file is used to store sensitive information such as API keys
var configurationBuilder = new ConfigurationBuilder()
    .SetBasePath(Directory.GetCurrentDirectory())
    .AddJsonFile("local.settings.json", optional: true, reloadOnChange: true)
    .AddJsonFile("secrets.settings.json", optional: true, reloadOnChange: true);
var config = configurationBuilder.Build();

// IMPORTANT: You ONLY NEED either Azure OpenAI or OpenAI connectiopn info, not both.
// Azure OpenAI Connection Info
var azureOpenAIEndpoint = config["AzureOpenAI:Endpoint"];
var azureOpenAIAPIKey = config["AzureOpenAI:APIKey"];
var azureOpenAIModelDeploymentName = config["AzureOpenAI:ModelDeploymentName"];
// OpenAI Connection Info 
var openAIAPIKey = config["OpenAI:APIKey"];
var openAIModelId = config["OpenAI:ModelId"];

Installed Packages Microsoft.Extensions.Configuration, 9.0.0 Microsoft.Extensions.Configuration.Json, 9.0.0

In [2]:
// Install the required NuGet packages
// Note: This also installs the Dependency Injection Package to retrieve the ChatCompletionService
#r "nuget: Microsoft.Extensions.DependencyInjection, 9.0.0"
#r "nuget: Microsoft.SemanticKernel, 1.33.0"

using Microsoft.Extensions.DependencyInjection.Extensions;
using Microsoft.Extensions.DependencyInjection;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;

Kernel semanticKernel;

// Set the flag to use Azure OpenAI or OpenAI. False to use OpenAI, True to use Azure OpenAI
var useAzureOpenAI = true;

// Create a new Semantic Kernel instance
if (useAzureOpenAI)
{
    Console.WriteLine("Using Azure OpenAI Service");
    semanticKernel = Kernel.CreateBuilder()
        .AddAzureOpenAIChatCompletion(
            deploymentName: azureOpenAIModelDeploymentName,
            endpoint: azureOpenAIEndpoint,
            apiKey: azureOpenAIAPIKey)
        .Build();
}
else
{
    Console.WriteLine("Using OpenAI Service");
    semanticKernel = Kernel.CreateBuilder()
        .AddOpenAIChatCompletion(
            modelId: openAIModelId,
            apiKey: openAIAPIKey)
        .Build();
}

var chatCompletionService = semanticKernel.Services.GetRequiredService<IChatCompletionService>();

Installed Packages Microsoft.Extensions.DependencyInjection, 9.0.0 Microsoft.SemanticKernel, 1.33.0

Using Azure OpenAI Service


### Step 2 - Decision Frames (Thinking Hats) Scenario for a Major Purchase 

The code below will list the potential decision frames (thinking hats) that may be appropriate to form different approaches for purchasing a house. Generative AI can be used to easily "put on a hat" (approach a problem with a specific decision frame) much easier than a human decision maker.  

In [3]:
// Set the overall system prompt to behave like a decision intelligence assistant (persona)
var systemPrompt = """
You are a decision intelligence assistant. 
Assist the user in exploring options, reasoning through decisions, problem-solving, and applying systems thinking to various scenarios. 
Provide structured, logical, and comprehensive advice.
""";

// Decision Frames Prompt that lists the six thinking hats approach to purchasing a house 
var decisionFramesPrompt = """
You are planning on purchasing a new home. 
Use Edward de Bono's Six thinking hats on how to approach the decision-making process. 

Instructions: 
List an approach for each of the six hats. 
""";

// Create a new chat history object with proper system and user message roles
var chatHistory = new ChatHistory();
chatHistory.AddSystemMessage(systemPrompt);
chatHistory.AddUserMessage(decisionFramesPrompt);

var chatResponse = string.Empty;
await foreach (var content in chatCompletionService.GetStreamingChatMessageContentsAsync(chatHistory))
{
    chatResponse += content;
    Console.Write(content);
}
chatHistory.AddAssistantMessage(chatResponse);

The Six Thinking Hats framework by Edward de Bono offers a structured way to explore a decision from multiple perspectives. Here's how you can apply the six hats to the decision-making process for purchasing a new home:

---

### 1. **White Hat (Facts and Information)**  
Focus on data, factual information, and gaps in knowledge.  

- **Approach:**  
   - Take inventory of what you know about the housing market, desired locations, current mortgage rates, inspection costs, and average housing prices.  
   - Research neighborhoods, schools, commute times, and crime rates if they are relevant to your decision.  
   - Identify specific information gaps (e.g., are property taxes higher in one area compared to another?) and seek answers.  
   - Assess pre-approval loan possibilities and budgeting details (savings for the down payment, additional closing costs).  
   - Create a checklist of the essential criteria for the home you want (size, features, age, etc.).

---

### 2. **Red Hat (Feeli

This can be enhanced further by having GenAI suggest which of the decision frames might be the most useful for a high-stakes decision. Therefore, the prompt above can be optimized further by listing all the decision frames and then priorotizing which ones will have the most impact and influence on the house purchase decision. 

In [7]:
// Set the overall system prompt to behave like a decision intelligence assistant (persona)
var systemPrompt = """
You are a decision intelligence assistant. 
Assist the user in exploring options, reasoning through decisions, problem-solving, and applying systems thinking to various scenarios. 
Provide structured, logical, and comprehensive advice.
""";

// Decision Frames Prompt that lists the six thinking hats approach to purchasing a house 
var decisionFramesListPrompt = """
You are planning on purchasing a new home. 
Use Edward de Bono's Six thinking hats on how to approach the decision-making process. 

Instructions:
List an approach for each of the six hats in the order of impact & influence from most to least. 
From the priorotized list suggest which thinking hats (decision frames) to focus on. 
It can be one of the hats or any combination of the thinking hats. 
""";

// Create a new chat history object with proper system and user message roles
var chatHistory = new ChatHistory();
chatHistory.AddSystemMessage(systemPrompt);
chatHistory.AddUserMessage(decisionFramesListPrompt);

var chatResponse = string.Empty;
await foreach (var content in chatCompletionService.GetStreamingChatMessageContentsAsync(chatHistory))
{
    chatResponse += content;
    Console.Write(content);
}
chatHistory.AddAssistantMessage(chatResponse);

When approaching the decision-making process of purchasing a new home, it's valuable to use Edward de Bono's Six Thinking Hats. Each hat represents a different type of thinking that can influence your decision. Let's prioritize the hats based on their impact and influence when making this significant decision:

1. **White Hat (Information and Facts)**
2. **Yellow Hat (Positive Judgment)**
3. **Black Hat (Critical Judgment)**
4. **Green Hat (Creativity)**
5. **Red Hat (Emotions and Intuition)**
6. **Blue Hat (Process and Control)**

### Prioritized List with Approaches

1. **White Hat (Information and Facts)**
   - **Approach:** Gather all available data regarding the property market, specific areas of interest, property prices, interest rates, demographics, local amenities, future development plans, and economic forecasts. Objective information forms the basis for a well-informed decision.

2. **Yellow Hat (Positive Judgment)**
   - **Approach:** Focus on the benefits and advantages of

So far, (given the house purchase decision) Generative AI has helped create decision frames with the six thinking hats framework, rank them from most impact & influence from most to least and select the top 3 decision frames. The next logical step is to leverage Generative AI to apply the selected 3 decision frames for the house purchase decision.  

This can be done in two different ways:  
1) **Single Decision Prompt:** Instruct (prompt) the GenAI model "think internally" on the decision frames, ranking, selecting the top decision frames and then finally output the applied decision frames for the house purchase decision.
2) **Workflow Decision Prompts:** This approach uses multiple prompts together to arrive at a decision approach. This breaks the problem down into smaller components and executes each component in sequence. The value is that each step can be optimized individually and each intermediate output is clearly available for analysis. This is ideal for sufficiently important decision problems.  

#### Single Decision Prompt 

In [ ]:
// Set the overall system prompt to behave like a decision intelligence assistant (persona)
var systemPrompt = """
You are a Decision Intelligence assistant. 
Assist the user in exploring options, reasoning through decisions, problem-solving, and applying systems thinking to various scenarios. 
Provide structured, logical, and comprehensive advice.
""";

// Decision Frames Prompt that lists the six thinking hats approach to purchasing a house 
var decisionFramesListPrompt = """
You are planning on purchasing a new home. 
Use Edward de Bono's Six thinking hats on how to approach the decision-making process. 

Instructions:

<Do Not Show these steps>
INTERNALLY THINK about an approach for each of the six hats in the order of impact & influence from most to least. 
From the priorotized list INTERNALLY PROCESS which thinking hats (decision frames) to focus on. 
It can be one of the hats or any combination of the thinking hats. 
Let's call that internal list "selected decision frames"
</Do Not Show these steps>

Only output these steps to the user: 
Finally, from the internal "selected decision frames", create an approach for each for purchasing a house.  
""";

// Create a new chat history object with proper system and user message roles
var chatHistory = new ChatHistory();
chatHistory.AddSystemMessage(systemPrompt);
chatHistory.AddUserMessage(decisionFramesListPrompt);

var chatResponse = string.Empty;
await foreach (var content in chatCompletionService.GetStreamingChatMessageContentsAsync(chatHistory))
{
    chatResponse += content;
    Console.Write(content);
}
chatHistory.AddAssistantMessage(chatResponse);

### Decision Frameworks for Purchasing a New Home (Based on Selected Decision Frames):

**1. White Hat (Focus on Facts and Information):**  
- Begin by gathering all available data on potential properties. This includes pricing, neighborhood characteristics, taxes, school districts, crime rates, and proximity to essential services like public transport and healthcare.  
- Research housing market trends for the area: Are prices rising, falling, or stable? This helps determine whether it’s an optimal time to buy.  
- Understand your financial situation by consulting your credit report, savings, loan pre-approval limits, and monthly budget for mortgage payments, utilities, and maintenance.  

**2. Yellow Hat (Optimistic Thinking – Benefits and Opportunities):**  
- Evaluate the positive aspects of buying a property. For example, consider the opportunity to build equity, secure a long-term asset, and customize your living space to meet your lifestyle needs.  
- Explore the future potential